# Liver interface

Just run this single cell. After some minutes you'll be able to use the generated interface for everything else.

In [1]:
#@title
%%capture --no-display 
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except: pass
environment = os.getenv("LIVER_ENVIRONMENT", "colab")

if environment == "local":
    drive_mount = "/mnt/chromeos/GoogleDrive"
elif environment == "colab":
    from google.colab import drive
    drive.mount("/content/drive")
    drive_mount="/content/drive"
    
    %cd /content
    !apt install elastix libgdcm-tools

    !git clone https://github.com/yamatteo/liver.git /content/liver
    !git clone https://github.com/AIM-Harvard/pyradiomics.git /content/radiomics

    %cd /content/liver 
    !git fetch origin july_rewrite 
    !git checkout july_rewrite
    !git pull origin july_rewrite
    !pip install -r requirements.txt
    
import components

state, tab = components.get_new_interface()

state.environment = environment
state.drive_folder = "COLAB"
state.drive_mount = drive_mount

display(tab)

In [ ]:
from importlib import reload
from pathlib import Path

import torch

import model.hunet, model.block, model.data
import report
from model.hunet import HunetNetwork
!wandb login 6e44b780601adbcb29c5322b462b5da80078222e

In [ ]:
reload(model.block)
reload(model.data)
reload(model.hunet)

dataset_path = Path("/data")
dataset_path.mkdir(exist_ok=True)
model.data.store_dataset(Path("/content/drive/MyDrive") / "COLAB", dataset_path, slice_shape=(512, 512, 40))


In [ ]:
reload(report)
reload(model.block)
reload(model.data)
reload(model.hunet)

dataset_path = Path("/data")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
nw = model.hunet.HunetNetwork(dataset_path, device)
models_path = Path("/content/liver/model/saved_models")
nw.train(models_path=models_path)

In [ ]:
# !git status
# !git config --global user.email "yamatteo@gmail.com"
# !git config --global user.name "Matteo Bortolotto"
# !git add .
# !git commit -m "debug"
# !git push https://github.com/yamatteo/liver.git